In [1]:
# importing libs
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, GaussianNoise, concatenate, Flatten, Conv1D,MaxPool1D
from keras.models import Model
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
import random as rn
import os
from keras.models import load_model

Using TensorFlow backend.


In [2]:
# Loading data
real_input=np.loadtxt('real_input.txt')
#print(real_input.shape)
Rinput=[]
for i in range(0,131072):
    Rinput.append(real_input[i][1])
Rinput=np.array(Rinput)
Rinput=np.reshape(Rinput,(16384,8))
#print(Rinput.shape)
#print(Rinput[2])

valinput=np.loadtxt('valinput.txt')
#print(valinput.shape)
Vinput=[]
for i in range(0,8192):
    Vinput.append(real_input[i][1])
Vinput=np.array(Vinput)
Vinput=np.reshape(Vinput,(1024,8))

real_output=np.loadtxt('real_output.txt')
#print(real_output.shape)
Routput=[]
for i in range(0,131072):
    Sum=real_output[4*i][1]+real_output[4*i+1][1]+real_output[4*i+2][1]+real_output[4*i+3][1]
    mm=Sum/4
    Routput.append(mm)
Routput=np.array(Routput)
Routput=np.reshape(Routput,(16384,8))
#print(Rinput.shape)
#print(Routput[2])

valoutput=np.loadtxt('valoutput.txt')
#print(valoutput.shape)
Voutput=[]
for i in range(0,8192):
    Sum=valoutput[4*i][1]+valoutput[4*i+1][1]+valoutput[4*i+2][1]+valoutput[4*i+3][1]
    mm=Sum/4
    Voutput.append(mm)
Voutput=np.array(Voutput)
Voutput=np.reshape(Voutput,(1024,8))

In [3]:
Ir=[0,1]
If=[1,0]

Ir1=[]
If1=[]
Ir2=[]
If2=[]
for i in range(0,16384):
    Ir1.append(Ir)
    If1.append(If)
    
for i in range(0,1024):
    Ir2.append(Ir)
    If2.append(If)
    
Ir1=np.array(Ir1)
Ir2=np.array(Ir2)
If1=np.array(If1)
If2=np.array(If2)

In [37]:
#Channel
Inputsize = 8
Outputsize = 8
input_signal = Input(shape=(Inputsize,))
lay1 = Dense(30, activation='relu')(input_signal)
lay2 = Dense(20, activation='relu')(lay1)
lay3 = Dense(13, activation='relu')(lay2)
lay4 = Dense(8, activation='relu')(lay3)
lay5 = Dense(5, activation='relu')(lay4)
output_signal = Dense(Outputsize, activation='linear')(lay5)

#generator = Model(input_signal, output_signal)

#generator.compile(optimizer='adam', loss='categorical_crossentropy')
#print (generator.summary())

#Discriminator
inputsize = 8
outputsize = 2
#input_sig = Input(shape=(inputsize,)

layer1 = Dense(16, activation='relu')(output_signal)
layer2 = Dense(10, activation='relu')(layer1)
layer3 = Dense(6, activation='relu')(layer2)
p = Dense(outputsize, activation='softmax')(layer3)

#discriminator = Model(input_sig, p)

#discriminator.compile(optimizer='adam', loss='categorical_crossentropy')
#print (discriminator.summary())

channel=Model(input_signal,p)
channel.compile(optimizer='adam', loss='categorical_crossentropy')
print (channel.summary())

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 8)                 0         
_________________________________________________________________
dense_89 (Dense)             (None, 30)                270       
_________________________________________________________________
dense_90 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_91 (Dense)             (None, 13)                273       
_________________________________________________________________
dense_92 (Dense)             (None, 8)                 112       
_________________________________________________________________
dense_93 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_94 (Dense)             (None, 8)                 48 

In [11]:
#Discriminator
input_sig = Input(shape=(inputsize,))
disc=channel.layers[-4](input_sig)
disc=channel.layers[-3](disc)
disc=channel.layers[-2](disc)
disc=channel.layers[-1](disc)
discriminator = Model(input_sig, disc)

discriminator.compile(optimizer='adam', loss='categorical_crossentropy')
print (discriminator.summary())

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 8)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 16)                144       
_________________________________________________________________
dense_18 (Dense)             (None, 10)                170       
_________________________________________________________________
dense_19 (Dense)             (None, 6)                 66        
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 14        
Total params: 394
Trainable params: 394
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
#Generator
generator = Model(input_signal, output_signal)

generator.compile(optimizer='adam', loss='categorical_crossentropy')
print (generator.summary())

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 8)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 30)                270       
_________________________________________________________________
dense_12 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_13 (Dense)             (None, 13)                273       
_________________________________________________________________
dense_14 (Dense)             (None, 8)                 112       
_________________________________________________________________
dense_15 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_16 (Dense)             (None, 8)                 48  

In [13]:
#初始化权重
keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)

In [44]:
#Training of discriminator
discriminator.fit(Routput, Ir1,
                epochs=50,
                batch_size=500,
                validation_data=(Voutput, Ir2))

Train on 16384 samples, validate on 1024 samples
Epoch 1/50
16384/16384 [==============================] - 0s 3us/step - loss: 3.2257e-05 - val_loss: 3.1948e-05
Epoch 2/50
16384/16384 [==============================] - 0s 3us/step - loss: 3.1700e-05 - val_loss: 3.1471e-05
Epoch 3/50
16384/16384 [==============================] - 0s 2us/step - loss: 3.1149e-05 - val_loss: 3.0875e-05
Epoch 4/50
16384/16384 [==============================] - 0s 3us/step - loss: 3.0614e-05 - val_loss: 3.0397e-05


C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 5/50
16384/16384 [==============================] - 0s 3us/step - loss: 3.0077e-05 - val_loss: 2.9802e-05
Epoch 6/50
16384/16384 [==============================] - 0s 2us/step - loss: 2.9557e-05 - val_loss: 2.9325e-05
Epoch 7/50
16384/16384 [==============================] - 0s 2us/step - loss: 2.9045e-05 - val_loss: 2.8808e-05
Epoch 8/50
16384/16384 [==============================] - 0s 2us/step - loss: 2.8534e-05 - val_loss: 2.8252e-05
Epoch 9/50
16384/16384 [==============================] - 0s 3us/step - loss: 2.8038e-05 - val_loss: 2.7775e-05
Epoch 10/50
16384/16384 [==============================] - 0s 3us/step - loss: 2.7548e-05 - val_loss: 2.7299e-05
Epoch 11/50
16384/16384 [==============================] - 0s 3us/step - loss: 2.7068e-05 - val_loss: 2.6822e-05
Epoch 12/50
16384/16384 [==============================] - 0s 2us/step - loss: 2.6595e-05 - val_loss: 2.6345e-05
Epoch 13/50
16384/16384 [==============================] - 0s 2us/step - loss: 2.6129e-05 - val_loss:

In [45]:
Foutput = generator.predict(Rinput) 
FVoutput=generator.predict(Vinput)

In [46]:
#Training of channel step 1
channel.layers[-1].trainable = False
channel.layers[-2].trainable = False
channel.layers[-3].trainable = False
channel.layers[-4].trainable = False
channel.fit(Foutput, If1,
                epochs=50,
                batch_size=500,
                validation_data=(FVoutput, If2))

Train on 16384 samples, validate on 1024 samples
Epoch 1/50
16384/16384 [==============================] - 0s 4us/step - loss: 7.3196e-09 - val_loss: 7.3342e-09
Epoch 2/50
16384/16384 [==============================] - 0s 3us/step - loss: 7.3196e-09 - val_loss: 7.3342e-09
Epoch 3/50
16384/16384 [==============================] - 0s 3us/step - loss: 7.3196e-09 - val_loss: 7.3342e-09
Epoch 4/50
16384/16384 [==============================] - 0s 3us/step - loss: 7.3196e-09 - val_loss: 7.3342e-09
Epoch 5/50
16384/16384 [==============================] - 0s 3us/step - loss: 7.3196e-09 - val_loss: 7.3342e-09
Epoch 6/50
16384/16384 [==============================] - 0s 3us/step - loss: 7.3196e-09 - val_loss: 7.3342e-09
Epoch 7/50
16384/16384 [==============================] - 0s 3us/step - loss: 7.3196e-09 - val_loss: 7.3342e-09
Epoch 8/50
16384/16384 [==============================] - 0s 3us/step - loss: 7.3196e-09 - val_loss: 7.3342e-09
Epoch 9/50
16384/16384 [==============================]

In [47]:
#Training of the channel
channel.layers[-1].trainable = False
channel.layers[-2].trainable = False
channel.layers[-3].trainable = False
channel.layers[-4].trainable = False
channel.fit(Rinput, Ir1,
                epochs=50,
                batch_size=500,
                validation_data=(Vinput, Ir2))

Train on 16384 samples, validate on 1024 samples
Epoch 1/50
16384/16384 [==============================] - 0s 4us/step - loss: 2.5640 - val_loss: 0.0133
Epoch 2/50
16384/16384 [==============================] - 0s 3us/step - loss: 0.0023 - val_loss: 3.5658e-04
Epoch 3/50
16384/16384 [==============================] - 0s 3us/step - loss: 2.5289e-04 - val_loss: 2.0542e-04
Epoch 4/50
16384/16384 [==============================] - 0s 3us/step - loss: 1.7170e-04 - val_loss: 1.5304e-04
Epoch 5/50
16384/16384 [==============================] - 0s 3us/step - loss: 1.3154e-04 - val_loss: 1.2018e-04
Epoch 6/50
16384/16384 [==============================] - 0s 3us/step - loss: 1.0564e-04 - val_loss: 9.8228e-05
Epoch 7/50
16384/16384 [==============================] - 0s 3us/step - loss: 8.7154e-05 - val_loss: 8.0970e-05
Epoch 8/50
16384/16384 [==============================] - 0s 3us/step - loss: 7.0650e-05 - val_loss: 6.2595e-05
Epoch 9/50
16384/16384 [==============================] - 0s 3us/st

In [48]:
Inp=Input(shape=(Inputsize,))
gener=channel.layers[-10](Inp)
gener=channel.layers[-9](gener)
gener=channel.layers[-8](gener)
gener=channel.layers[-7](gener)
gener=channel.layers[-6](gener)
gener=channel.layers[-5](gener)

Generator=Model(Inp, gener)
Generator.compile(optimizer='adam', loss='categorical_crossentropy')
print(Generator.summary())

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 8)                 0         
_________________________________________________________________
dense_89 (Dense)             (None, 30)                270       
_________________________________________________________________
dense_90 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_91 (Dense)             (None, 13)                273       
_________________________________________________________________
dense_92 (Dense)             (None, 8)                 112       
_________________________________________________________________
dense_93 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_94 (Dense)             (None, 8)                 48 

In [49]:
from keras.models import load_model
Generator.save('channel_generator.model')